In [64]:
from src.RawDataLoaders import CAsT_RawDataLoader
from src.models_and_transforms.text_transforms import Rewriter_Query_Resolver_Transform, Rewriter_Context_Query_Merge_Transform, \
                                                      BART_Numericalise_Transform, Rewriter_Context_Target_Transform, Query_Cleaner_Transform
from src.models_and_transforms.complex_transforms import BART_Query_Rewriter_Transform, BART_Full_Conversational_Rewriter_Transform
from src.pipe_datasets import CAsT_Query_ReWriting_Dataset
from src.models_and_transforms.BART_models import BART_Query_ReWriter
from src.trainers import Model_Trainer

from transformers import BertTokenizer, BartTokenizer
from pytorch_lightning import Trainer, Callback, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
import torch
from tqdm.auto import tqdm 
import  random
import os
import jsonlines
import numpy as np
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [2]:
seed_everything(42)

42

In [44]:
raw_data_loader = CAsT_RawDataLoader()
get_query_fn = raw_data_loader.get_query
get_doc_fn = raw_data_loader.get_doc

In [17]:
train_raw_samples = raw_data_loader.get_topics("train", ignore_missing_q_rels=True)

In [18]:
len(train_raw_samples)

376

In [6]:
[1,2,3][:1]

[1]

In [19]:
train_samples = Rewriter_Query_Resolver_Transform(get_query_fn)(train_raw_samples)
train_samples = Rewriter_Context_Query_Merge_Transform()(train_samples)
train_samples = Rewriter_Context_Target_Transform()(train_samples)
train_samples = BART_Numericalise_Transform(fields=[('input_text', 'input_ids'), ('target_text', 'target_ids')])(train_samples)

In [20]:
train_dataset = CAsT_Query_ReWriting_Dataset(train_samples, get_query_fn, merge_mode="full_context_rewrite")

In [10]:
train_dataset[4]

{'prev_turns': ['31_1', '31_2', '31_3', '31_4'],
 'q_id': '31_5',
 'q_rel': ['MARCO_2509296'],
 'unresolved_query': 'Can it spread to the throat?',
 'previous_queries': ['What is throat cancer?',
  'Is throat cancer treatable?',
  'Tell me about lung cancer.',
  "What are lung cancer's symptoms?"],
 'resolved_query': 'Can lung cancer spread to the throat?',
 'input_text': "What is throat cancer? Is throat cancer treatable? Tell me about lung cancer. What are lung cancer's symptoms? query: Can it spread to the throat?",
 'target_text': "What is throat cancer? Is throat cancer treatable? Tell me about lung cancer. What are lung cancer's symptoms? query: Can lung cancer spread to the throat?",
 'input_ids': [0,
  2264,
  16,
  14599,
  1668,
  116,
  1534,
  14599,
  1668,
  3951,
  868,
  116,
  11378,
  162,
  59,
  10665,
  1668,
  4,
  653,
  32,
  10665,
  1668,
  18,
  5298,
  116,
  25860,
  35,
  2615,
  24,
  2504,
  7,
  5,
  14599,
  116,
  2],
 'target_ids': [0,
  2264,
  16,


In [11]:
train_dataloader = train_dataset.to_dataloader(16)

In [9]:
batch = list(iter(train_dataloader))[17]

In [10]:
os.path.join

{'input_ids': tensor([[    0, 35438,   162,  ...,     1,     1,     1],
         [    0, 35438,   162,  ...,     1,     1,     1],
         [    0, 25860,    35,  ...,     1,     1,     1],
         ...,
         [    0,  2264,    16,  ...,     1,     1,     1],
         [    0,  2264,    16,  ...,     1,     1,     1],
         [    0,  2264,    16,  ...,     1,     1,     1]]),
 'input_attention_mask': tensor([[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.]]),
 'decoder_input_ids': tensor([[    0, 35438,   162,  ...,     1,     1,     1],
         [    0, 35438,   162,  ...,     1,     1,     1],
         [    0, 25860,    35,  ...,     1,     1,     1],
         ...,
         [    0,  2264,    16,  ...,     1,     1,     1],
         [    0,  2264,    16,  ...,     1,     1,     1],
 

In [13]:
val_raw_samples = raw_data_loader.get_topics("dev", ignore_missing_q_rels=True)
val_dataset = CAsT_Query_ReWriting_Dataset(val_raw_samples, get_query_fn, merge_mode="full_context_rewrite")
val_dataloader = val_dataset.to_dataloader(32)

In [14]:
len(val_dataset)

103

In [28]:
model = BART_Query_ReWriter()

Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-large-cnn and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(os.getcwd(),"saved_models/BART_Rewriter/"),
    save_top_k=3,
    verbose=True,
    monitor='val_loss',
    mode='max',
    prefix=''
)

In [30]:
trainer = Trainer(gpus=1, gradient_clip_val=0.5, check_val_every_n_epoch=0.5,checkpoint_callback=checkpoint_callback)
trainer.fit(model, train_dataloader, val_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name | Type                         | Params
------------------------------------------------------
0 | BART | BartForConditionalGeneration | 406 M 
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


{'BLEU': 0.25714959565972084}
-----EXAMPLE-0------
Input Text: 'What are mammals? What are mammals' key characteristics? How many legs do mammals have? What is the largest mammal on land? What is the largest mammal in the world? What is the largest mammal in the UK? What about the largest mammal to ever walk the earth? Tell me about Blue whales. query: How much do they weigh?'
      Pred: 'What are mammals? What are mammals' key characteristics? How many legs do mammals have? What is the largest mammal on land? What about the largest mammals to ever walk the earth? Tell me about Blue whales. How much do they weigh? What do they look like? How big are they?'
    Target: ' How much do Blue whales weigh?'
------------------
-----EXAMPLE-1------
Input Text: 'What are important literary elements? query: What is the difference between them and literary devices?'
      Pred: ' What is the difference between them and literary devices? What are the differences between literary devices and liter

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: The metric you returned 0.3599902714572068 must be a `torch.Tensor` instance, checkpoint not saved HINT: what is the value of val_loss in validation_epoch_end()?
  warnings.warn(*args, **kwargs)

Epoch 00000: val_loss reached 0.35999 (best 0.35999), saving model to /nfs/phd_by_carlos/notebooks/saved_models/BART_Rewriter/epoch=0.ckpt as top 3


{'BLEU': 0.3599902714572068}
-----EXAMPLE-0------
Input Text: 'What are important literary elements? What is the difference between literary elements and literary devices? What is a literary genre? What are the most important literary genres? What is a trope? What are important examples of tropes in fantasy? What is the definition of allegory? What are some important modern uses of allegory? query: What are the different types of foreshadowing?'
      Pred: 'What are important literary elements and literary devices? What is a literary genre? What are the most important literary genres? What's a trope and what are important examples of tropes in fantasy? How are the different types of foreshadowing used in fantasy novels? How is allegory defined and used in modern fiction?'
    Target: ' What are the different types of foreshadowing?'
------------------
-----EXAMPLE-1------
Input Text: 'What is taught in sociology? What is the main contribution of Auguste Comte to sociology? What is the

...

{'BLEU': 0.5585010273455056}
-----EXAMPLE-0------
Input Text: 'What are the pros and cons of electric cars? Tell me more about Tesla the car company. Why did Elon Musk start Tesla? What Tesla car models are available? What is the best selling Tesla car model? What is unique about the Tesla Model 3? What are the safety features of Tesla Model 3? Why is Tesla building Gigafactories? What makes the Tesla batteries unique? How long can Tesla car batteries last? query: Tell me more about autopilot.'
      Pred: ' Tell me about autopilot.'
    Target: ' Tell me more about autopilot in Tesla cars.'
------------------
-----EXAMPLE-1------
Input Text: 'What do Spanish people do on Christmas day? query: What do they eat for dinner?'
      Pred: ' What do Spanishpeople eat for dinner on Christmas Day? ' ?",Registrationroad Theenseem "anga!",'
    Target: ' What do Spanish people eat for Christmas dinner?'
------------------
-----EXAMPLE-2------
Input Text: 'What's the difference between soup and 

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: The metric you returned 0.5635661023605504 must be a `torch.Tensor` instance, checkpoint not saved HINT: what is the value of val_loss in validation_epoch_end()?
  warnings.warn(*args, **kwargs)

Epoch 00066: val_loss  was not in top 3


{'BLEU': 0.5635661023605504}
-----EXAMPLE-0------
Input Text: 'Who is the most successful pirate of all time? Who is the most famous female pirate? Tell me more about Ching Shih. What were Ching Shih's code of laws? query: How did she die?'
      Pred: ' How did ChingShih the pirate of the 1800s die?'
    Target: ' How did Ching Shih die?'
------------------
-----EXAMPLE-1------
Input Text: ' query: Who is the most successful pirate of all time?'
      Pred: ' Who is the most successful pirate of all time?  The?", 'roadRegistration!",angaemsyMagicenseetchendone "'
    Target: ' Who is the most successful pirate of all time?'
------------------
-----EXAMPLE-2------
Input Text: 'What are mammals? What are mammals' key characteristics? How many legs do mammals have? What is the largest mammal on land? What is the largest mammal in the world? What is the largest mammal in the UK? What about the largest mammal to ever walk the earth? Tell me about Blue whales. How much do Blue whales weigh?

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [32]:
torch.save(model.state_dict(), "saved_models/BART_Rewriter/BART_save_dict.ckpt")

In [35]:
model.load_from_checkpoint(checkpoint_path="saved_models/BART_Rewriter/BART_save_dict.ckpt")

Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-large-cnn and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyError: 'state_dict'

In [13]:
my_trainer = Model_Trainer(gpus=[3])
nan_batch = my_trainer.train(model, train_dataloader)

Detected 8 GPUS available, using [3].



Keyboard Interrupt!


In [28]:
BART_query_rewriter_transform = BART_Query_Rewriter_Transform("saved_models/BART_Rewriter/BART_save_dict.ckpt")

Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-large-cnn and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT ReRanker initialised on cuda. Batch size 1


In [37]:
test_samples = [{'unresolved_query':'how do they compare?', 'previous_queries':['How do you know when your cable is going bad?',
                                                                               'Now it stopped working. Why?',
                                                                               'how much does a lock cost?',
                                                                               'can they be hacked?']}]
test_samples = BART_query_rewriter_transform(test_samples)
test_samples = Query_Cleaner_Transform(fields=[('rewritten_query','clean_rewritten_query')])(test_samples)

In [38]:
test_samples[0]["clean_rewritten_query"]

' how do lock types compare to a cable and cable go bad?'

In [59]:
eval_raw_samples = raw_data_loader.get_topics("all", ignore_missing_q_rels=True)
eval_raw_samples = Rewriter_Query_Resolver_Transform(get_query_fn, prev_queries_utter_type="raw_utterance")(eval_raw_samples)

In [60]:
len(eval_raw_samples)

479

In [62]:
eval_raw_samples[4]

{'prev_turns': ['31_1', '31_2', '31_3', '31_4'],
 'q_id': '31_5',
 'q_rel': ['MARCO_2509296'],
 'unresolved_query': 'Can it spread to the throat?',
 'previous_queries': ['What is throat cancer?',
  'Is it treatable?',
  'Tell me about lung cancer.',
  'What are its symptoms? '],
 'resolved_query': 'Can lung cancer spread to the throat?',
 'full_rewritten_queries': [' What is throat cancer?',
  ' Is throat cancer treatable?',
  ' Tell me about lung cancer. What is throat cancer?',
  " What are lung cancer's symptoms?",
  ' Can lung cancer spread to the throat?'],
 'rewritten_query': ' Can lung cancer spread to the throat?'}

In [103]:
eval_raw_samples = BART_query_rewriter_transform(eval_raw_samples)
eval_raw_samples = Query_Cleaner_Transform(fields=[('rewritten_query','clean_rewritten_query')])(eval_raw_samples)

In [104]:
for sample in eval_raw_samples:
    del sample["q_rel"]
    del sample["input_ids"]
    del sample["pred_ids"]

In [105]:
eval_raw_samples[4]

{'prev_turns': ['31_1', '31_2', '31_3', '31_4'],
 'q_id': '31_5',
 'unresolved_query': 'Can it spread to the throat?',
 'previous_queries': ['What is throat cancer?',
  'Is throat cancer treatable?',
  'Tell me about lung cancer.',
  "What are lung cancer's symptoms?"],
 'resolved_query': 'Can lung cancer spread to the throat?',
 'input_text': "What is throat cancer? Is throat cancer treatable? Tell me about lung cancer. What are lung cancer's symptoms? query: Can it spread to the throat?",
 'rewritten_query': 'What is throat cancer? Is throat cancer treatable? Tell me about lung cancer. What are lung cancer\'s symptoms? query: Can lung cancer spread to the throat? The \'\xa0?",Registrationem and',
 'clean_rewritten_query': ' Can lung cancer spread to the throat?'}

In [63]:
import csv
keys = eval_raw_samples[0].keys()
with open('saved_models/BART_Rewriter/BART_y1_self_rewrites.csv', 'w', newline='')  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(eval_raw_samples)

In [55]:
BART_conv_transform = BART_Full_Conversational_Rewriter_Transform("saved_models/BART_Rewriter/BART_save_dict.ckpt")

Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-large-cnn and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT ReRanker initialised on cuda. Batch size 1


In [61]:
test_samples = [{'unresolved_query':'How do they compare?', 'previous_queries':['How do you know when your skateboard is going bad?',
                                                                               'Now it stopped working. Why?',
                                                                               'how much does a lock cost?',
                                                                               'can they be hacked?']}]
eval_raw_samples = BART_conv_transform(eval_raw_samples)

In [65]:
with jsonlines.open('BART_y1_self_rewrites.jsonl', mode='w') as writer:
    writer.write(eval_raw_samples)